In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#read in the excel file
df = pd.read_excel("c:\\Users\\595217\\Downloads\\All_DLPT.xls", sheet_name="Export Worksheet")

#rename columns to lower case
df.columns = [x.lower() for x in df.columns]

#convert completion date to datatime format
df['completiondate'] = pd.to_datetime(df['completiondate'], infer_datetime_format=True)

df.describe()

,ld_sid,lang_event_type,completiondate,grade,lang_descr,lang_type
count,27681,27681,27670,27634,27681,27681
unique,2909,4,2238,11,68,2
top,11A2066,DLPT_LISTENING,2014-08-20 00:00:00,3,SPANISH,SELECTED
freq,49,12260,61,13644,7819,27136
first,NaN,NaN,1915-01-21 00:00:00,NaN,NaN,NaN
last,NaN,NaN,2018-11-27 00:00:00,NaN,NaN,NaN


In [3]:
#remove records where grade is empty (NaN)
df = df[(df.grade.notnull()) & (df.lang_type == 'SELECTED')]

In [4]:
#select just "listening" and "reading" lang_event_type
df = df[(df.lang_event_type == 'DLPT_LISTENING') | (df.lang_event_type == 'DLPT_READING')]

In [5]:
#create variable for score and an empty column for score in the df
score, df['score'] = [], ''

#iterate through the df and append numerical scores to the score variable
for row in df.itertuples(index=True, name='Pandas'):
    if getattr(row, "grade") == "4+":
        score.append(4.5)
    elif getattr(row, "grade") == "3+":
        score.append(3.5)
    elif getattr(row, "grade") == "2+":
        score.append(2.5)
    elif getattr(row, "grade") == "1+":
        score.append(1.5)
    elif getattr(row, "grade") == "0+":
        score.append(.5)
    else:
        score.append(getattr(row, "grade"))

#append the score variable to the df.score column
df['score'] = score

#convert the df.score column to numeric
df['score'] = pd.to_numeric(df['score'])

#drop records where completion date is null
df = df[df.completiondate.notnull()]

#sort the df
df = df.sort_values(by=['ld_sid', 'lang_descr', 'lang_event_type', 'completiondate']).reset_index(drop=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23914 entries, 0 to 23913
Data columns (total 7 columns):
ld_sid             23914 non-null object
lang_event_type    23914 non-null object
completiondate     23914 non-null datetime64[ns]
grade              23914 non-null object
lang_descr         23914 non-null object
lang_type          23914 non-null object
score              23914 non-null float64
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.3+ MB


In [6]:
#check the df to be sure there are no null values present
df.isna().any()

ld_sid             False
lang_event_type    False
completiondate     False
grade              False
lang_descr         False
lang_type          False
score              False
dtype: bool

In [7]:
#create a new df which counts the number of scores per candidate
dfg = df.groupby(['lang_event_type', 'ld_sid', 'lang_descr'])['score'].agg(['count', 'first', 'mean', 'std']).reset_index()
dfg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5773 entries, 0 to 5772
Data columns (total 7 columns):
lang_event_type    5773 non-null object
ld_sid             5773 non-null object
lang_descr         5773 non-null object
count              5773 non-null int64
first              5773 non-null float64
mean               5773 non-null float64
std                4961 non-null float64
dtypes: float64(3), int64(1), object(3)
memory usage: 315.8+ KB


In [8]:
#create a df where there is more than one score for a candidate
morethanonetest = dfg[dfg['count'] > 1]
morethanonetest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4961 entries, 0 to 5772
Data columns (total 7 columns):
lang_event_type    4961 non-null object
ld_sid             4961 non-null object
lang_descr         4961 non-null object
count              4961 non-null int64
first              4961 non-null float64
mean               4961 non-null float64
std                4961 non-null float64
dtypes: float64(3), int64(1), object(3)
memory usage: 310.1+ KB


In [9]:
#create a df of just the first score for each candidate
first_scores = df.groupby(['lang_event_type', 'lang_descr', 'ld_sid'])['score'].first().reset_index()
first_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5773 entries, 0 to 5772
Data columns (total 4 columns):
lang_event_type    5773 non-null object
lang_descr         5773 non-null object
ld_sid             5773 non-null object
score              5773 non-null float64
dtypes: float64(1), object(3)
memory usage: 180.5+ KB


In [10]:
#create a df of the means of all but the first score for each candidate
def mask_first(x):
    result = np.ones_like(x)
    result[0] = 0
    return result

mask = df.groupby(['lang_event_type', 'lang_descr', 'ld_sid'])['ld_sid'].transform(mask_first).astype(bool)
mean_scores = df.loc[mask].groupby(['lang_event_type', 'lang_descr', 'ld_sid'])['score'].mean().reset_index()
mean_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4961 entries, 0 to 4960
Data columns (total 4 columns):
lang_event_type    4961 non-null object
lang_descr         4961 non-null object
ld_sid             4961 non-null object
score              4961 non-null float64
dtypes: float64(1), object(3)
memory usage: 155.1+ KB


In [11]:
#merge the dataframes together
merged = pd.merge(first_scores, mean_scores, how='inner', on=['ld_sid', 'lang_event_type', 'lang_descr'])

In [12]:
#pivot the table to show listening and reading scores per individual and drop records that are missing scores
both_scores = merged.pivot_table(index=['ld_sid', 'lang_descr'], columns='lang_event_type', values=['score_x', 'score_y']).dropna()

In [13]:
len(both_scores)

2433

In [14]:
#remove any 3/3 or above on First DLPT
both_scores = both_scores[(both_scores['score_x']['DLPT_LISTENING'] <= 2.5) | (both_scores['score_x']['DLPT_READING'] <= 2.5)]

In [15]:
#count of unique IDs
both_scores.reset_index()['ld_sid'].nunique()

1628

In [16]:
#define functions to create columns for status

def listening(df):
    for i, row in df.iterrows():
        ifor_val = "error"
        if row['score_x']['DLPT_LISTENING'] == row['score_y']['DLPT_LISTENING']:
            ifor_val = "maintained"
        if row['score_x']['DLPT_LISTENING'] < row['score_y']['DLPT_LISTENING']:
            ifor_val = "improved"
        if row['score_x']['DLPT_LISTENING'] > row['score_y']['DLPT_LISTENING']:
            ifor_val = "declined"
        df.at[i,'listening_status'] = ifor_val

def reading(df):
    for i, row in df.iterrows():
        ifor_val = "error"
        if row['score_x']['DLPT_READING'] == row['score_y']['DLPT_READING']:
            ifor_val = "maintained"
        if row['score_x']['DLPT_READING'] < row['score_y']['DLPT_READING']:
            ifor_val = "improved"
        if row['score_x']['DLPT_READING'] > row['score_y']['DLPT_READING']:
            ifor_val = "declined"
        df.at[i,'reading_status'] = ifor_val
    
def overall(df):
    for i, row in df.iterrows():
        ifor_val = "error"
        if ((row['listening_status'] == "improved") & (row['reading_status'] == "improved")).bool():
            ifor_val = "improved"
        elif ((row['listening_status'] == "declined") & (row['reading_status'] == "declined")).bool():
            ifor_val = "declined"
        elif ((row['listening_status'] == "maintained") & (row['reading_status'] == "maintained")).bool():
            ifor_val = "maintained"
        elif ((row['listening_status'] == "maintained") & (row['reading_status'] == "improved")).bool():
            ifor_val = "improved"
        elif ((row['listening_status'] == "improved") & (row['reading_status'] == "maintained")).bool():
            ifor_val = "improved"
        elif ((row['listening_status'] == "maintained") & (row['reading_status'] == "declined")).bool():
            ifor_val = "declined"
        elif ((row['listening_status'] == "declined") & (row['reading_status'] == "maintained")).bool():
            ifor_val = "declined"
        else:
            ifor_val = "mixed"
        df.at[i, 'overall_status'] = ifor_val

In [17]:
#initiate the functions and check the df
listening(both_scores)
reading(both_scores)
overall(both_scores)
both_scores.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1628 entries, (10A1003, SWAHILI) to (18E0006, FRENCH)
Data columns (total 7 columns):
(score_x, DLPT_LISTENING)    1628 non-null float64
(score_x, DLPT_READING)      1628 non-null float64
(score_y, DLPT_LISTENING)    1628 non-null float64
(score_y, DLPT_READING)      1628 non-null float64
(listening_status, )         1628 non-null object
(reading_status, )           1628 non-null object
(overall_status, )           1628 non-null object
dtypes: float64(4), object(3)
memory usage: 276.7+ KB


In [18]:
both_scores.head()

score_x                     score_y  \
lang_event_type          DLPT_LISTENING DLPT_READING DLPT_LISTENING   
ld_sid  lang_descr                                                    
10A1003 SWAHILI                     1.5          2.0       2.312500   
10A1005 FRENCH                      2.5          2.5       2.285714   
10A1006 CHINESE MANDARIN            0.5          0.0       0.375000   
10A1008 SPANISH                     2.0          2.5       2.111111   
10A1010 JAPANESE                    1.5          0.5       1.500000   

                                      listening_status reading_status  \
lang_event_type          DLPT_READING                                   
ld_sid  lang_descr                                                      
10A1003 SWAHILI              2.562500         improved       improved   
10A1005 FRENCH               2.722222         declined       improved   
10A1006 CHINESE MANDARIN     0.125000         declined       improved   
10A1008 SPANISH              2.388889         improved       declined   
10A1010 JAPANESE             1.400000       maintained       improved   

                         overall_status  
lang_event_type                          
ld_sid  lang_descr                       
10A1003 SWAHILI                improved  
10A1005 FRENCH                    mixed  
10A1006 CHINESE MANDARIN          mixed  
10A1008 SPANISH                   mixed  
10A1010 JAPANESE               improved

In [19]:
both_scores.columns = [' '.join(col).strip().lower() for col in both_scores.columns.values]
both_scores.columns = ['first_dlpt_listening', 'first_dlpt_reading', 'mean_dlpt_listening', 'mean_dlpt_reading', 'listening_status',
       'reading_status', 'overall_status']
both_scores = both_scores.reset_index()
both_scores.columns

Index(['ld_sid', 'lang_descr', 'first_dlpt_listening', 'first_dlpt_reading',
       'mean_dlpt_listening', 'mean_dlpt_reading', 'listening_status',
       'reading_status', 'overall_status'],
      dtype='object')

In [20]:
#read in the excel file
ementor = pd.read_excel("c:\\Users\\595217\\Downloads\\ementor_report.xlsx", parse_dates=['STARTDATE', 'ENDDATE'])

#rename columns to lower case
ementor.columns = [x.lower() for x in ementor.columns]

ementor.describe()

,ld_sid,rank,firstname,lastname,lang_descr,coursenumber,startdate,enddate,provider,language_type,groups
count,4398,4398,4398,4398,4398,4398,4398,4398,4398,4398,4398
unique,2299,16,1129,1843,64,3032,1130,1148,2,2,9
top,11A2063,Capt,MICHAEL,KIM,SPANISH,QB17-3012M1,2018-02-05 00:00:00,2017-07-27 00:00:00,PEC,Selected,LEAP
freq,8,1729,89,68,1173,6,27,22,3804,4332,4022
first,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-16 00:00:00,2013-11-20 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-15 00:00:00,2018-11-24 00:00:00,NaN,NaN,NaN


In [21]:
#read in the excel file
lite = pd.read_excel("c:\\Users\\595217\\Downloads\\LITE_report.xlsx", parse_dates=['travel_date', 'return_date', 'confirm_date'])

#rename columns to lower case
lite.columns = [x.lower() for x in lite.columns]

lite.describe()

,lite_leg,leap_level_score
count,3540.000000,2628.000000
mean,1.161582,3.303881
std,0.572413,0.716970
min,1.000000,0.800000
25%,1.000000,3.000000
50%,1.000000,3.600000
75%,1.000000,3.800000
max,6.000000,4.000000


In [22]:
#select only participants that completed both an ementor or a lite program
final = both_scores[both_scores['ld_sid'].isin(np.append(ementor['ld_sid'].unique(), lite['ld_sid'].unique()))]

In [23]:
#note we may need to merge dataframes to ensure both ld_sid and lang_desc match.

In [24]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1492 entries, 0 to 1605
Data columns (total 9 columns):
ld_sid                  1492 non-null object
lang_descr              1492 non-null object
first_dlpt_listening    1492 non-null float64
first_dlpt_reading      1492 non-null float64
mean_dlpt_listening     1492 non-null float64
mean_dlpt_reading       1492 non-null float64
listening_status        1492 non-null object
reading_status          1492 non-null object
overall_status          1492 non-null object
dtypes: float64(4), object(5)
memory usage: 116.6+ KB


In [25]:
final

,ld_sid,lang_descr,first_dlpt_listening,first_dlpt_reading,mean_dlpt_listening,mean_dlpt_reading,listening_status,reading_status,overall_status
0,10A1003,SWAHILI,1.5,2.0,2.312500,2.562500,improved,improved,improved
1,10A1005,FRENCH,2.5,2.5,2.285714,2.722222,declined,improved,mixed
2,10A1006,CHINESE MANDARIN,0.5,0.0,0.375000,0.125000,declined,improved,mixed
3,10A1008,SPANISH,2.0,2.5,2.111111,2.388889,improved,declined,mixed
4,10A1010,JAPANESE,1.5,0.5,1.500000,1.400000,maintained,improved,improved
5,10A1014,RUSSIAN,0.5,0.5,0.875000,0.312500,improved,declined,mixed
6,10A1015,PORTUGUESE BRAZILIAN,2.0,2.5,2.928571,3.000000,improved,improved,improved
7,10A1018,JAPANESE,1.5,1.5,1.642857,1.750000,improved,improved,improved
8,10A1020,ARABIC MODERN STANDARD,2.5,2.5,2.937500,2.937500,improved,improved,improved
9,10A1022,CHINESE MANDARIN,1.0,1.0,1.250000,1.000000,improved,maintained,improved
